Initial Purpose: take Kuprov algorithm (which is much slower than mine) and try to make it faster than mine with cython.

Now: Maybe I can fully vectorize with numpy!

In [1]:
%load_ext cython
import Cython
print(Cython.__version__)

import numpy as np


0.29.6


In [2]:
from simulation_data import spin8

Testing indicated, surprisingly, that Kuprov algorithm ~~had almost identical speeds with lil_matrix, csr_matrix, *and* regular numpy dense matrices.~~ Update: actually, switching to np.kron made the dense version ca 30x *faster* somehow! So, won't worry with cython and scipy integration, just cython and numpy.

Kuprov version of hamiltonian:

In [3]:
def kuprov_H(v, J):
    sigma_x = np.matrix([[0, 1 / 2], [1 / 2, 0]])
    sigma_y = np.matrix([[0, -1j / 2], [1j / 2, 0]])
    sigma_z = np.matrix([[1 / 2, 0], [0, -1 / 2]])
    unit = np.matrix([[1, 0], [0, 1]])

    nspins = len(v)
    Lx = []
    Ly = []
    Lz = []

    for n in range(nspins):
        Lx_current = 1
        Ly_current = 1
        Lz_current = 1

        for k in range(nspins):
            if k == n:
                Lx_current = np.kron(Lx_current, sigma_x)
                Ly_current = np.kron(Ly_current, sigma_y)
                Lz_current = np.kron(Lz_current, sigma_z)
            else:
                Lx_current = np.kron(Lx_current, unit)
                Ly_current = np.kron(Ly_current, unit)
                Lz_current = np.kron(Lz_current, unit)

        Lx.append(Lx_current)
        Ly.append(Ly_current)
        Lz.append(Lz_current)
#     print('test kuprov')
#     print(Lx[0])
#     print(Lx[1])
#     print(Lx[0]*Lx[1])
#     print(Lx[0]*Lx[1] + Ly[0]*Ly[1] + Lz[0]*Lz[1])
#     print('Kuprov')
#     print('Lx: ')
#     print(Lx)
#     print('Ly: ')
#     print(Ly)
#     print('Lz: ')
#     print(Lz)
        
    H = np.zeros((2**nspins, 2**nspins), dtype=np.complex128)
    for n in range(nspins):
        H += v[n] * Lz[n]

    for n in range(nspins):
        for k in range(nspins):
            if n != k:
                H += 0.5 * J[n, k] * (Lx[n]*Lx[k] + Ly[n]*Ly[k] + Lz[n]*Lz[k])

    return H

In [4]:
dense_result = %timeit -o kuprov_H(*spin8())
dense_result.average


422 ms ± 71.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


0.4221517571427934

In [7]:
DTYPE = np.complex128

Let's try dividing the function into smaller functions. Also, let's try defining a single matrix for stacked Lx, Ly, and Lz.

In [8]:
def spin_operators(nspins):
    sigma_x = np.array([[0, 1 / 2], [1 / 2, 0]])
    sigma_y = np.array([[0, -1j / 2], [1j / 2, 0]])
    sigma_z = np.array([[1 / 2, 0], [0, -1 / 2]])
    unit = np.array([[1, 0], [0, 1]])

#     Lx = []
#     Ly = []
#     Lz = []
    L = np.empty((3, nspins, 2**nspins, 2**nspins), dtype=DTYPE)
#     print(L.shape)
    for n in range(nspins):
        Lx_current = 1
        Ly_current = 1
        Lz_current = 1

        for k in range(nspins):
            if k == n:
                Lx_current = np.kron(Lx_current, sigma_x)
                Ly_current = np.kron(Ly_current, sigma_y)
                Lz_current = np.kron(Lz_current, sigma_z)
            else:
                Lx_current = np.kron(Lx_current, unit)
                Ly_current = np.kron(Ly_current, unit)
                Lz_current = np.kron(Lz_current, unit)

#         Lx.append(Lx_current)
#         Ly.append(Ly_current)
#         Lz.append(Lz_current)
        L[0][n] = Lx_current
        L[1][n] = Ly_current
        L[2][n] = Lz_current
#     print('mine')
#     print('Lx: ')
#     print(L[0])
#     print('Ly: ')
#     print(L[1])
#     print('Lz: ')
#     print(L[2])
        
    return L

In [9]:
L = spin_operators(2)
L[1]

array([[[0.+0.j , 0.+0.j , 0.-0.5j, 0.+0.j ],
        [0.+0.j , 0.+0.j , 0.+0.j , 0.-0.5j],
        [0.+0.5j, 0.+0.j , 0.+0.j , 0.+0.j ],
        [0.+0.j , 0.+0.5j, 0.+0.j , 0.+0.j ]],

       [[0.+0.j , 0.-0.5j, 0.+0.j , 0.-0.j ],
        [0.+0.5j, 0.+0.j , 0.+0.j , 0.+0.j ],
        [0.+0.j , 0.-0.j , 0.+0.j , 0.-0.5j],
        [0.+0.j , 0.+0.j , 0.+0.5j, 0.+0.j ]]])

In [10]:
def hamiltonian(v, J, L):
#     H = np.zeros((2**nspins, 2**nspins), dtype=np.complex128)
#     for n in range(nspins):
#         H += v[n] * Lz[n]
    nspins = len(v)
    Lx = L[0]
    Ly = L[1]
    Lz = L[2]
#     print('test mine')
#     print(Lx[0])
#     print(Lx[1])
#     print(Lx[0]@Lx[1])
#     print(Lx[0]*Lx[1] + Ly[0]*Ly[1] + Lz[0]*Lz[1])
#     print(L[0][0])
#     print(L[0][1])
#     print(L[0][0]*L[0][1])
    H = np.tensordot(v, L[2], axes=1)

    for n in range(nspins):
        for k in range(nspins):
            if n != k:
#                 H += 0.5 * J[n, k] * (L[0][n]*L[0][k] + L[1][n]*L[1][k] + L[2][n]*L[2][k])
                H += 0.5 * J[n, k] * (Lx[n]@Lx[k] + Ly[n]@Ly[k] + Lz[n]@Lz[k])

    return H
    

In [11]:
m1 = np.array([[0, 1],[2, 3]])
m2 = np.array([[10, 20], [30, 40]])
a = np.array([m1, m2])
v = np.array([2, 4])
np.tensordot(v, a, axes=1)

array([[ 40,  82],
       [124, 166]])

In [12]:
v = np.array([142.5, 157.5])
J = np.array([[0, 12], [12, 0]])
L = spin_operators(2)
H1 = kuprov_H(v, J)
H2 = hamiltonian(v, J, L)
print(H1)
print(H2)

[[ 153. +0.j    0. +0.j    0. +0.j    0. +0.j]
 [   0. +0.j  -10.5+0.j    6. +0.j    0. +0.j]
 [   0. +0.j    6. +0.j    4.5+0.j    0. +0.j]
 [   0. +0.j    0. +0.j    0. +0.j -147. +0.j]]
[[ 153. +0.j    0. +0.j    0. +0.j    0. +0.j]
 [   0. +0.j  -10.5+0.j    6. +0.j    0. +0.j]
 [   0. +0.j    6. +0.j    4.5+0.j    0. +0.j]
 [   0. +0.j    0. +0.j    0. +0.j -147. +0.j]]


In [13]:
np.array_equal(H1, H2)

True

In [14]:
L

array([[[[ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ],
         [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ],
         [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ],
         [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ]],

        [[ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ],
         [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ],
         [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ],
         [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ]]],


       [[[ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ],
         [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j],
         [ 0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ],
         [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ]],

        [[ 0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ],
         [ 0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ],
         [ 0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.5j],
         [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ]]],


       [[[ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ],
         [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ]

Test of math for Cartesian product of spin operators. 
Source: https://stackoverflow.com/questions/47752324/matrix-multiplication-on-4d-numpy-arrays

In [15]:
M1 = np.full((2, 2), 1)
M2 = np.full((2, 2), 2)
M3 = np.full((2, 2), 3)
M4 = np.full((2, 2), 4)
M5 = np.full((2, 2), 5)
M6 = np.full((2, 2), 6)

In [16]:
M = np.array([[M1, M2],[M3, M4], [M5, M6]])  # the shape of a 2-spin spin operator matrix
print(M.shape)

(3, 2, 2, 2)


In [17]:
M_T = M.transpose(1, 0, 2, 3)
print(M_T.shape)
print(M_T)

(2, 3, 2, 2)
[[[[1 1]
   [1 1]]

  [[3 3]
   [3 3]]

  [[5 5]
   [5 5]]]


 [[[2 2]
   [2 2]]

  [[4 4]
   [4 4]]

  [[6 6]
   [6 6]]]]


In [18]:
M.T

array([[[[1, 3, 5],
         [2, 4, 6]],

        [[1, 3, 5],
         [2, 4, 6]]],


       [[[1, 3, 5],
         [2, 4, 6]],

        [[1, 3, 5],
         [2, 4, 6]]]])

In [19]:
prod = np.tensordot(M_T, M, axes=((1,3),(0,2))).swapaxes(1,2)
prod.shape

(2, 2, 2, 2)

In [20]:
prod


array([[[[ 70,  70],
         [ 70,  70]],

        [[ 88,  88],
         [ 88,  88]]],


       [[[ 88,  88],
         [ 88,  88]],

        [[112, 112],
         [112, 112]]]])

In [21]:
def hamiltonian_2(v, J, L):
    nspins = len(v)
#     Lx = L[0]
#     Ly = L[1]
#     Lz = L[2]

    H = np.tensordot(v, L[2], axes=1)
    L_T = L.transpose(1, 0, 2, 3)
    Lproduct = np.tensordot(L_T, L, axes=((1,3),(0,2))).swapaxes(1,2)
    scalars = 0.5 * J
#     scalars = np.multiply(scalars, Lproduct)
#     for n in range(nspins):
#         for k in range(nspins):
#             H += scalars[n, k].real
#     off_diagonal = np.tensordot(scalars, Lproduct, axes=2)
#     print(off_diagonal.size)
#     print(off_diagonal)
#     H += off_diagonal
    H += np.tensordot(scalars, Lproduct, axes=2)
    return H

In [22]:
H3 = hamiltonian_2(v, J, L)
H3

array([[ 153. +0.j,    0. +0.j,    0. +0.j,    0. +0.j],
       [   0. +0.j,  -10.5+0.j,    6. +0.j,    0. +0.j],
       [   0. +0.j,    6. +0.j,    4.5+0.j,    0. +0.j],
       [   0. +0.j,    0. +0.j,    0. +0.j, -147. +0.j]])

In [23]:
np.array_equal(H1, H3)

True

In [24]:
from simulation_data import spin8

In [25]:
vtest, jtest = spin8()
Ltest = spin_operators(8)

In [26]:
kuprov_result = %timeit -o kuprov_H(vtest, jtest)
print('kuprov: ', kuprov_result.average)

417 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
kuprov:  0.4167508000000453


In [27]:
H_result = %timeit -o hamiltonian(vtest, jtest, Ltest)
print('hamiltonian: ', H_result.average)

455 ms ± 48.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
hamiltonian:  0.45512449999997834


In [28]:
H2_result = %timeit -o hamiltonian_2(vtest, jtest, Ltest)
print('H2: ', H2_result.average)

356 ms ± 78.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
H2:  0.35587209999968245


In [29]:
result = np.matmul(M_T, M)
print(result.shape)
print(result)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,3,2,2)->(2,3,newaxis,newaxis) (3,2,2,2)->(3,2,newaxis,newaxis) and requested shape (2,2)

In [30]:

L_T = L.transpose(1, 0, 2, 3) 
print(L.shape, L_T.shape)
Lprod = np.matmul(L_T, L)
# Lprod = L.transpose(1, 0, 2, 3)@L
Lprod.shape

(3, 2, 4, 4) (2, 3, 4, 4)


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,3,4,4)->(2,3,newaxis,newaxis) (3,2,4,4)->(3,2,newaxis,newaxis) and requested shape (4,4)

In [31]:
def hamiltonian2(v, J, L):
#     H = np.zeros((2**nspins, 2**nspins), dtype=np.complex128)
#     for n in range(nspins):
#         H += v[n] * Lz[n]
    nspins = len(v)
    Lx = L[0]
    Ly = L[1]
    Lz = L[2]
#     print('test mine')
#     print(Lx[0])
#     print(Lx[1])
#     print(Lx[0]@Lx[1])
#     print(Lx[0]*Lx[1] + Ly[0]*Ly[1] + Lz[0]*Lz[1])
#     print(L[0][0])
#     print(L[0][1])
#     print(L[0][0]*L[0][1])
    H = np.tensordot(v, L[2], axes=1)
    Lprod = L.transpose(1, 0, 2, 3)@L
#     H += 0.5 * J.reshape(2**nspins) * 
    for n in range(nspins):
        for k in range(nspins):
            if n != k:
#                 H += 0.5 * J[n, k] * (L[0][n]*L[0][k] + L[1][n]*L[1][k] + L[2][n]*L[2][k])
                H += 0.5 * J[n, k] * (Lx[n]@Lx[k] + Ly[n]@Ly[k] + Lz[n]@Lz[k])

    return H

In [32]:
v, J = spin8()
L = spin_operators(8)
H1 = kuprov_H(v, J)
H2 = hamiltonian(v, J, L)
print(H1)
print(H2)


[[ 741.25+0.j    0.  +0.j    0.  +0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 [   0.  +0.j  473.75+0.j    6.  +0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 [   0.  +0.j    6.  +0.j  500.25+0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 ...
 [   0.  +0.j    0.  +0.j    0.  +0.j ... -494.75+0.j    6.  +0.j
     0.  +0.j]
 [   0.  +0.j    0.  +0.j    0.  +0.j ...    6.  +0.j -471.25+0.j
     0.  +0.j]
 [   0.  +0.j    0.  +0.j    0.  +0.j ...    0.  +0.j    0.  +0.j
  -723.75+0.j]]
[[ 741.25+0.j    0.  +0.j    0.  +0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 [   0.  +0.j  473.75+0.j    6.  +0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 [   0.  +0.j    6.  +0.j  500.25+0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 ...
 [   0.  +0.j    0.  +0.j    0.  +0.j ... -494.75+0.j    6.  +0.j
     0.  +0.j]
 [   0.  +0.j    0.  +0.j    0.  +0.j ...    6.  +0.j -471.25+0.j
     0.  +0.j]
 [   0.  +0.j    0.  +0.j    0.  +0.j ...    0.  +0.j    0.  +0.j
  -723.75+0.j]]


In [33]:
%%cython -a
import numpy as np

def c1(v, J):
    sigma_x = np.matrix([[0, 1 / 2], [1 / 2, 0]])
    sigma_y = np.matrix([[0, -1j / 2], [1j / 2, 0]])
    sigma_z = np.matrix([[1 / 2, 0], [0, -1 / 2]])
    unit = np.matrix([[1, 0], [0, 1]])

    cdef Py_ssize_t nspins = len(v)
    Lx = []
    Ly = []
    Lz = []
    
    cdef Py_ssize_t n, k

    for n in range(nspins):
        Lx_current = 1
        Ly_current = 1
        Lz_current = 1

        for k in range(nspins):
            if k == n:
                Lx_current = np.kron(Lx_current, sigma_x)
                Ly_current = np.kron(Ly_current, sigma_y)
                Lz_current = np.kron(Lz_current, sigma_z)
            else:
                Lx_current = np.kron(Lx_current, unit)
                Ly_current = np.kron(Ly_current, unit)
                Lz_current = np.kron(Lz_current, unit)

        Lx.append(Lx_current)
        Ly.append(Ly_current)
        Lz.append(Lz_current)

    H = np.zeros((2**nspins, 2**nspins), dtype=np.complex128)
    for n in range(nspins):
        H += v[n] * Lz[n]

    for n in range(nspins):
        for k in range(nspins):
            if n != k:
                H += 0.5 * J[n, k] * (Lx[n]*Lx[k] + Ly[n]*Ly[k] + Lz[n]*Lz[k])

    return H

DistutilsPlatformError: Unable to find vcvarsall.bat

In [34]:
from speedtest.kuprov import kuprov_H_dense

In [35]:
dense_result = %timeit -o kuprov_H_dense(*spin8())
dense_result.average

467 ms ± 70.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


0.4667303428572528

In [36]:
a = np.array([[1, 7], [4, 3]]) 
b = np.array([[2, 9], [4, 5]]) 
c = np.array([[3, 6], [1, 0]]) 
d = np.array([[2, 8], [1, 2]]) 
e = np.array([[0, 0], [1, 2]])
f = np.array([[2, 8], [1, 0]])

m = np.array([[a, b], [c, d]])              # (2,2,2,2)
n = np.array([[e, f, a], [b, d, c]])        # (2,3,2,2)

In [37]:
mn = np.tensordot(m,n, axes=([1,3],[0,2]))

In [38]:
np.array([[a@e+b@b, a@f+b@d, a@a+b@c],
          [c@e+d@b, c@f+d@d, c@a+d@c]])

array([[[[47, 77],
         [31, 67]],

        [[22, 42],
         [24, 74]],

        [[44, 40],
         [33, 61]]],


       [[[42, 70],
         [10, 19]],

        [[24, 56],
         [ 6, 20]],

        [[41, 51],
         [ 6, 13]]]])

In [39]:
mn

array([[[[47, 77],
         [22, 42],
         [44, 40]],

        [[31, 67],
         [24, 74],
         [33, 61]]],


       [[[42, 70],
         [24, 56],
         [41, 51]],

        [[10, 19],
         [ 6, 20],
         [ 6, 13]]]])